<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_08/LangGraph%20Agent%20Design%20Patterns/Pattern_1_Prompt_Chaining_(LangGraph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Step 1: Install required libraries

In [1]:
!pip install -U langgraph langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 26.7 MB/s eta 0:00:00


## Step 2: Set Environment for LangSmith Tracing

In [3]:
import os
import getpass

#  Prompt user to enter keys securely (not visible in UI)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_PROJECT"] = "prompt-chaining-email"
os.environ["LANGCHAIN_TRACING_V2"] = "true"


Enter your OpenAI API key: ··········
Enter your LangSmith API key: ··········


## Step 3: Define Prompt Chaining Nodes & Graph

In [6]:
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

# Step 3: Define prompt chaining graph

def summarize_complaint(state):
    input_text = state["input"]
    summary = f"This is a summary of the complaint: {input_text}"
    return {"summary": summary}

def generate_email(state):
    summary = state["summary"]
    email = f"""
Dear Customer,

Thank you for reaching out. Regarding your concern: "{summary}" —
we’re looking into the matter and will follow up shortly.

Best regards,
Customer Support Team
"""
    return {"email": email}

# Wrap with RunnableLambda
summarizer_node = RunnableLambda(summarize_complaint)
email_writer_node = RunnableLambda(generate_email)

# Create graph
builder = StateGraph(dict)
builder.add_node("summarize", summarizer_node)
builder.add_node("generate_email", email_writer_node)

builder.set_entry_point("summarize")
builder.add_edge("summarize", "generate_email")
builder.add_edge("generate_email", END)

workflow = builder.compile()


##  Step 4: Invoke Workflow and View LangSmith Trace

In [7]:
from pprint import pprint

# Test input for customer support email
input_data = {
    "input": "I ordered a keyboard last week but it still hasn’t arrived. Can you check the shipping status?"
}

#  Run the graph
result = workflow.invoke(input_data)

#  Final Output
pprint(result)


{'email': '\n'
          'Dear Customer,\n'
          '\n'
          'Thank you for reaching out. Regarding your concern: "This is a '
          'summary of the complaint: I ordered a keyboard last week but it '
          'still hasn’t arrived. Can you check the shipping status?" —\n'
          'we’re looking into the matter and will follow up shortly.\n'
          '\n'
          'Best regards,\n'
          'Customer Support Team\n'}


## Step 5: View LangSmith Trace for This Run

In [9]:
from langsmith import Client

client = Client()
runs = client.list_runs(project_name="prompt-chaining-email", limit=1)
latest_run = next(runs)

print(f"🔍 View Trace: https://smith.langchain.com/public/{latest_run.id}/projects")


🔍 View Trace: https://smith.langchain.com/public/47723299-6bc8-4b09-9785-05cbd7e494a3/projects
